# Installing python packages into Notebook

In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install oyaml
!{sys.executable} -m pip install pandas

In [ ]:
import api

In [ ]:
import imp
imp.reload(api)

# Speciific App

In [ ]:
app = api.App("prod", "dmzvm")

In [ ]:
api.LinkTable(app.iterlinks(), api.f.endpoint("zone", "prod"))

# Runtime Review

In [ ]:
run = api.Runtime()

In [ ]:
api.Runtime.zones

## Load up data

In [ ]:
%%time
run = run.refresh()

## Summary Stats

In [ ]:
api.Table(run.stats())

### Alert Count

In [ ]:
sum([a["Num Links"] for a in run.stats()])

## All the links

In [ ]:
api.LinkTable(run.iterlinks())

In [ ]:
for zone in run.iterzones():
    for app in zone.iterapps():
        for link in app.iterlinks():
            print(link)

# Meta Policies and Suggestions

In [ ]:
api.mpr.run(run.iterlinks()).review()

## Pending Ones

In [ ]:
api.LinkTable(run.iterlinks(),
              api.f.nstate(None),
              #api.f.endpoint("zone", "nightly-k8s"), api.f.endpoint("app", "^monitoring\."),
              #api.f.any(api.f.same_zone, api.f.ltype(["NAI", "NAE"])),
              #api.f.neg(api.f.endpoint("process", "grafana-server", who="client")),
              #api.f.neg(api.f.same_zone), api.f.ltype(["AEG", "AIN"]),
              #api.f.same_zone, api.f.ltype(["AEG", "AIN"]),
              #api.f.neg(api.f.ltype(["AEG", "AIN"]), api.f.same_zone),
              #api.f.perimeter,
              #api.f.neg(api.f.perimeter),
              #api.f.neg(api.f.server_non_ip),
              #api.f.server_non_ip
             )

## Trying a new Meta Policy

In [ ]:
class MpTest:                                                                                                                                  
    policies = [                                                                
        api.AcceptLink(filters=[                                                    
                api.f.endpoint("app", "bendvm.bend.backend", who="client"),               
                api.f.endpoint("process", "rsn/araali_backend.py", who="client"),              
                api.f.endpoint("app", "cassandra", who="server"),               
                api.f.endpoint("process", "org.apache.cassandra.service.CassandraDaemon", who="server"),              
            ], changes=[                                                        
            ]),                                                                                                                                
    ]    

In [ ]:
api.MetaPolicyRunner(MpTest).run(run.iterlinks()).review(MpTest)

## Analyzing Greened Ones

In [ ]:
api.LinkTable(run.iterlinks(),
              api.f.neg(api.f.nstate(None)),
              api.f.ltype("NAE"),
             )

# Relocation Example

In [ ]:
api.LinkTable(run.iterzones("nightly").iterapps("bendvm").relocate("prod", "bendvm").iterlinks())

In [ ]:
api.LinkTable(run.iterzones("nightly").iterapps("bendvm").to_lib().iterlinks())

# Documentation

## Meta-Policy HowTo

Meta policy are policies that are in turn used to accept policies (with or without changes and transformations). Araali policies are least permissive and precise, but they can be auto-accepted using patterns encoded into these meta policies. While accepting the policy, the endpoint parameters can be changed/rewritten (for client or server params in the policy).

Using the info displayed for a link in Alert state, create a meta policy using the following guidelines depending on the link type:
 * NAE
   * First find out if the process is already in some other meta-policy block you may have defined
     * It might be simple enough to add another dns into an existing meta policy (first example).
     * If there is change to the default suggested policy, you should not use a list for dns_pattern, and instead have one block per dns_pattern that is rewritten
       * In the first example below, if you put \*.pythonhosted.\*, it will end up using that even for links that match pypi.org
       * Instead use the separate block as shown in the second example
   * Else define a new block.
     * Typically you would want to have client app, process, and dns_pattern (to keep things zone independent) in the link filters.
<pre>        
    AcceptLink(filters=[                                                    
            f.ltype("NAE"),                                                 
            f.endpoint("app", "cassandra"),                                 
            f.endpoint("process", "/usr/bin/pip3", who="client"),           
            f.endpoint("dns_pattern", [":pypi.python.org:",                 
                                       ":pypi.org:",                        
                                       ":files.pythonhosted.org:"], who="server"),
        ], changes=[                                                        
        ]),
    AcceptLink(filters=[                                                    
            f.endpoint("process", ["/usr/bin/yum", "amazon_linux_extras"], who="client"),
            f.endpoint("dns_pattern", ":amazonlinux.us-west-2.amazonaws.com:", who="server"),
        ], changes=[                                                        
            ("server", "dns_pattern", ":amazonlinux\..*\.amazonaws\.com:"), 
        ]),
</pre>            
  * NAI
    * see if the service really needs public internet exposure. These are easy to spot with the pre-defined perimeter filter.
    * Define a block with f.perimeter filter and the process and app that needs perimeter exposure
    * Araali by default exposes these to the world, but you can restrict/change it to a subnet instead.
<pre>        
    AcceptLink(filters=[                                                    
            f.perimeter,                                                    
            f.endpoint("zone", ["prod", "nightly", "dev", "ops", "nightly-k8s"], who="server"),
            f.endpoint("app", ["dmzvm", "cassandra", "k8s"], who="server", flags=re.IGNORECASE), 
            f.endpoint("process", ["sshd", "haproxy"], who="server", flags=re.IGNORECASE),
        ], changes=[
            ("client", "network", "192.168.0.0"),
            ("client", "mask", 16),
        ]),
</pre>            
            
  * INT
    * There is no need to specify zone, because internal implies it by default
<pre>
    AcceptLink(filters=[                                                    
        f.ltype("INT"),                                                 
        f.endpoint("app", "monitoring.grafana.grafana"),          
        f.endpoint("process", "grafana-server", who="client"),          
        f.endpoint("process", "prometheus", who="server"),              
    ], changes=[                                                        
    ]),
</pre>            
  * AIN
    * These can be ommitted
      * Covering AEG automatically covers this category.
  * AEG
    * If both client and server should be in the same zone, use a same_zone filter, else both client and server zones can be explicitly specified (cross zone case). The example below uses same_zone.
<pre>
    AcceptLink(filters=[                                                    
            f.same_zone,                                                    
            f.ltype("AEG"),                                                 
            f.endpoint("app", "dmzvm", who="client"),                       
            f.endpoint("process", "/var/lib/haproxy/healthcheck.py", who="client"),
            f.endpoint("app", "bendvm.bend.backend", who="server"),         
            f.endpoint("process", "prometheus", who="server"),              
        ], changes=[                                                        
        ]),
</pre>            